In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Data Science Learning\\Projects\\End-to-end-ML-Project-with-MLflow'

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

         
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir)
            return data_ingestion_config

In [7]:
import os
import  urllib.request as request 
import zipfile 
from src.mlProject import logger 
from src.mlProject.utils.common import get_size 

In [8]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-23 15:35:59,148: INFO: common: yaml file: C:\Data Science Learning\Projects\End-to-end-ML-Project-with-MLflow\config\config.yaml loaded successfully]
[2025-01-23 15:35:59,154: INFO: common: yaml file: C:\Data Science Learning\Projects\End-to-end-ML-Project-with-MLflow\params.yaml loaded successfully]
[2025-01-23 15:35:59,157: INFO: common: yaml file: C:\Data Science Learning\Projects\End-to-end-ML-Project-with-MLflow\schema.yaml loaded successfully]
[2025-01-23 15:35:59,159: INFO: common: Created Directory at: artifacts]
[2025-01-23 15:35:59,160: INFO: common: Created Directory at: artifacts/data_ingestion]
[2025-01-23 15:35:59,161: INFO: 1556077380: File already exists of size: 23 KB]
